In [4]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import matplotlib.pyplot as plt
import re

In [5]:
from zipfile import ZipFile
with ZipFile('news_7k.zip', 'r') as zf:
    zf.extractall('./')

In [6]:
news_df = pd.read_csv('./news.csv')
news_df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(news_df, news_df, test_size = 0.2, random_state = 0)
#print(X_train, X_test, y_train, y_test)
X_train

,Unnamed: 0,title,text,label
2402,1551,Why Putin praised Trump (Opinion),Christian Whiton is a former deputy special en...,REAL
1922,1168,Super Tuesday Brings Harsh Light And Heartache...,Super Tuesday Brings Harsh Light And Heartache...,REAL
3475,7265,Reduce Nurse Burnout by Treating Nurses as Wel...,Prev post Page 1 of 4 Next \nNurses are among ...,FAKE
6197,3976,"Dozens dead, including one American, as hostag...",The deadly hostage situation at a luxury hotel...,REAL
4748,5298,Our new country: Women and minorities hit hardest,Our new country: Women and minorities hit hard...,FAKE
...,...,...,...,...
4931,3384,Many Clinton charity donors also got State Dep...,Twenty-two of the 37 corporations nominated fo...,REAL
3264,4551,Possibility of fire aboard EgyptAir flight rai...,"As pieces of luggage, human remains, wreckage ...",REAL
1653,8004,"World Proud Of Its Calm, Measured Response To ...",0 Add Comment \nIN THE immediate aftermath of ...,FAKE
2607,6264,780 Palestinian homes razed in occupied W Bank...,Palestine Palestinians check the flat of Amjad...,FAKE


In [8]:
df_news_train = X_train[['title','text']]
df_news_train_target = X_train[['label']]
df_news_test = X_test[['title','text']]
df_news_test_target =X_test[['label']]

In [9]:
!pip install wordcloud
from wordcloud import STOPWORDS
STOPWORDS.add("rt")
STOPWORDS.add("s")
STOPWORDS.add("u")
STOPWORDS.add("amp")
STOPWORDS.add("th")
STOPWORDS.add("will")
STOPWORDS.add("t")
STOPWORDS.add("m")

In [10]:

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\Mahima\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiword

True

In [11]:
train_df=df_news_train.reset_index(drop=True)
#df2.reindex_like(df1)
test_df =df_news_test.reset_index(drop=True)

In [12]:
import re
for i in range(len(train_df)):
    d = train_df.loc[i]['title']
    d=re.sub(r'http\S+', '', d) #remove links
    d=re.sub("[^a-zA-Z]", " ", d) #remove all characters except letters
    d=d.lower() #convert all words to lowercase
    d=nltk.word_tokenize(d) #split sentences into word
    d=[word for word in d if not word in STOPWORDS] #remove the stopwords
    lemma=nltk.WordNetLemmatizer() 
    d=[lemma.lemmatize(word) for word in d] #identify the correct form of the word in the dictionary
    d=" ".join(d)
    
    train_df.at[i,'title']=d

In [13]:
#Cleaning text data 
import re
for i in range(len(train_df)):
    d = train_df.loc[i]["text"]
    d=re.sub(r'http\S+', '', d) #remove links
    d=re.sub("[^a-zA-Z]", " ", d) #remove all characters except letters
    d=d.lower() #convert all words to lowercase
    d=nltk.word_tokenize(d) #split sentences into word
    d=[word for word in d if not word in STOPWORDS] #remove the stopwords
    lemma=nltk.WordNetLemmatizer() 
    d=[lemma.lemmatize(word) for word in d] #identify the correct form of the word in the dictionary
    d=" ".join(d)
    
    train_df.at[i,"text"]=d

In [14]:
train_target_df=df_news_train_target.reset_index(drop=True)
#df2.reindex_like(df1)
test_target_df =df_news_test_target.reset_index(drop=True)

In [15]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

def token(text):
    txt = nltk.word_tokenize(text.lower())
    return [st.stem(word) for word in txt]

In [16]:
cv = CountVectorizer(lowercase=True, 
                     tokenizer=token, stop_words=STOPWORDS,
                     analyzer=u'word', min_df=1)

In [17]:
tv = TfidfVectorizer(stop_words = STOPWORDS,lowercase=True, binary = True)

In [87]:
train_df

,title,text
0,putin praised trump opinion,christian whiton former deputy special envoy h...
1,super tuesday brings harsh light heartache mor...,super tuesday brings harsh light heartache mor...
2,reduce nurse burnout treating nurse well treat...,prev post page next nurse among underappreciat...
3,dozen dead including one american hostage situ...,deadly hostage situation luxury hotel mali cap...
4,new country woman minority hit hardest,new country woman minority hit hardest ann cou...
...,...,...
5063,many clinton charity donor got state departmen...,twenty two corporation nominated prestigious s...
5064,possibility fire aboard egyptair flight raised...,piece luggage human remains wreckage tell tale...
5065,world proud calm measured response trump victory,add comment immediate aftermath donald trump s...
5066,palestinian home razed occupied w bank area c,palestine palestinian check flat amjad aliwi i...


In [18]:
vec_train = tv.fit_transform(train_df['text'])
vec_test = tv.transform(test_df)

C:\Users\Mahima\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [19]:
def numeric_label(label_value):
    if(label_value == 'FAKE'):
        return 0
    else:
        return 1
        

In [20]:
train_target_df['target']=train_target_df['label'].apply(numeric_label)
train_target_df

,label,target
0,REAL,1
1,REAL,1
2,FAKE,0
3,REAL,1
4,FAKE,0
...,...,...
5063,REAL,1
5064,REAL,1
5065,FAKE,0
5066,FAKE,0


In [21]:
vec_train


<5068x52324 sparse matrix of type '<class 'numpy.float64'>'
	with 1345203 stored elements in Compressed Sparse Row format>

In [92]:
vec_test = tv.transform(test_df['text'])

In [104]:
print(vec_train)

  (0, 3128)	0.07441355570653506
  (0, 17655)	0.05033687298284524
  (0, 35345)	0.06730067591142708
  (0, 34853)	0.044537964271433736
  (0, 27062)	0.03170690521502001
  (0, 34195)	0.04516918383061473
  (0, 13243)	0.024917498029426312
  (0, 16803)	0.05529414116123271
  (0, 30509)	0.056913271726193815
  (0, 11081)	0.04775631638241476
  (0, 47146)	0.061155514595426015
  (0, 29610)	0.03852853178719326
  (0, 5899)	0.05661111143923509
  (0, 29107)	0.08893086148913669
  (0, 43428)	0.04363286004838752
  (0, 32046)	0.033148145727571666
  (0, 1414)	0.030103996029616973
  (0, 9980)	0.023990195701636893
  (0, 4611)	0.04036827295595432
  (0, 15464)	0.021433548225562087
  (0, 34420)	0.03831192522744751
  (0, 22938)	0.04248317775606784
  (0, 43676)	0.06858521493621558
  (0, 15754)	0.052666670425098296
  (0, 9508)	0.0344102341711364
  :	:
  (5067, 50418)	0.034405230341819415
  (5067, 44128)	0.05385275538958374
  (5067, 32097)	0.028357690177192963
  (5067, 18636)	0.048637003104577116
  (5067, 47160)	0.05

In [22]:
test_target_df['target']=test_target_df['label'].apply(numeric_label)
test_target_df

,label,target
0,REAL,1
1,FAKE,0
2,FAKE,0
3,FAKE,0
4,REAL,1
...,...,...
1262,REAL,1
1263,REAL,1
1264,FAKE,0
1265,REAL,1


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report


rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rfc.fit(X = vec_train, y = train_target_df['target'])

y_pred_rfc=rfc.predict(vec_test)
#accuracy_score(y_pred_rfc, test_target)


In [96]:
rfc_report=classification_report(test_target_df['target'],y_pred_rfc)
print(rfc_report)

              precision    recall  f1-score   support

           0       0.89      0.88      0.88       615
           1       0.89      0.90      0.89       652

    accuracy                           0.89      1267
   macro avg       0.89      0.89      0.89      1267
weighted avg       0.89      0.89      0.89      1267



In [97]:
train_df

,title,text
0,putin praised trump opinion,christian whiton former deputy special envoy h...
1,super tuesday brings harsh light heartache mor...,super tuesday brings harsh light heartache mor...
2,reduce nurse burnout treating nurse well treat...,prev post page next nurse among underappreciat...
3,dozen dead including one american hostage situ...,deadly hostage situation luxury hotel mali cap...
4,new country woman minority hit hardest,new country woman minority hit hardest ann cou...
...,...,...
5063,many clinton charity donor got state departmen...,twenty two corporation nominated prestigious s...
5064,possibility fire aboard egyptair flight raised...,piece luggage human remains wreckage tell tale...
5065,world proud calm measured response trump victory,add comment immediate aftermath donald trump s...
5066,palestinian home razed occupied w bank area c,palestine palestinian check flat amjad aliwi i...


In [99]:
bool_series_t = pd.notnull(train_df['text'])
train_df[bool_series_t]

,title,text
0,putin praised trump opinion,christian whiton former deputy special envoy h...
1,super tuesday brings harsh light heartache mor...,super tuesday brings harsh light heartache mor...
2,reduce nurse burnout treating nurse well treat...,prev post page next nurse among underappreciat...
3,dozen dead including one american hostage situ...,deadly hostage situation luxury hotel mali cap...
4,new country woman minority hit hardest,new country woman minority hit hardest ann cou...
...,...,...
5063,many clinton charity donor got state departmen...,twenty two corporation nominated prestigious s...
5064,possibility fire aboard egyptair flight raised...,piece luggage human remains wreckage tell tale...
5065,world proud calm measured response trump victory,add comment immediate aftermath donald trump s...
5066,palestinian home razed occupied w bank area c,palestine palestinian check flat amjad aliwi i...
